In [2]:
import glob

# All files and directories ending with .txt and that don't begin with a dot:
print(glob.glob("../chosen_data/crosswalk/zip_tract/*2020.xlsx"))

['../chosen_data/crosswalk/zip_tract\\zip_tract_032020.xlsx', '../chosen_data/crosswalk/zip_tract\\zip_tract_062020.xlsx', '../chosen_data/crosswalk/zip_tract\\zip_tract_092020.xlsx', '../chosen_data/crosswalk/zip_tract\\zip_tract_122020.xlsx']


In [54]:
import glob
import pandas as pd
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# To Create Modular
# Place in class
# Allow for different file schemas (Local, S3?)
# Maybe load into duckdb/spark/databricks and then proces the start and end date from the file path?

## Reads from disk in the expected format <type>_<end_month><year>.xlsx
def load_hud_crosswalk_from_local(local_path, year):
    # Find all crosswalk files in the directory
    files_to_load = glob.glob(f"{local_path}/*{year}.xlsx")

    # Build dataframes with start and end dates
    dfs = []
    pattern = r".*(\d{2})(?=\d{4}\.xlsx$)"
    for file in files_to_load:

        # Pull the month out of the expected file format
        match = re.search(pattern, file)
        if not match:
            raise ValueError("No month found in the file path")
        
        # Build the start and end date based on how the crosswalk data is written
        month = match.group(1)
        date = datetime.strptime(f"{year}-{month}-01", "%Y-%m-%d")
        start_date = date - relativedelta(months=2)

        if month == "12":
            end_date = date.replace(month=12, day=31)
        else:
            end_date = date.replace(month=date.month+1, day=1) - timedelta(days=1)
        
        # Load the XLSX, apply the start and end dates. Union, and Return
        df = pd.read_excel(file)
        df['start_date'] = start_date.strftime("%Y-%m-%d")
        df['end_date'] = end_date.strftime("%Y-%m-%d")
        dfs += [df]
    df_ret = pd.concat(dfs, axis=0, ignore_index=True)
    return df_ret

In [55]:
from IPython.display import display, HTML

loaded_zip_tract_df = pd.concat([load_hud_crosswalk_from_local("../chosen_data/crosswalk/zip_tract/", 2020), load_hud_crosswalk_from_local("../chosen_data/crosswalk/zip_tract/", 2022)], axis=0, ignore_index=True)
loaded_cbsa_zip_df = pd.concat([load_hud_crosswalk_from_local("../chosen_data/crosswalk/cbsa_zip/", 2020), load_hud_crosswalk_from_local("../chosen_data/crosswalk/cbsa_zip/", 2022)], axis=0, ignore_index=True)
display(loaded_zip_tract_df)
display(loaded_cbsa_zip_df)


d:\NASA_Interview\housing_data\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\NASA_Interview\housing_data\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\NASA_Interview\housing_data\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\NASA_Interview\housing_data\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\NASA_Interview\housing_data\.venv\Lib\site-packages\openp